# Auditoria de Dados Pessoais - LGPD

Este notebook demonstra técnicas de identificação, classificação e anonimização de dados pessoais (PII) em conformidade com a Lei Geral de Proteção de Dados.

## Objetivos

1. Gerar dados de exemplo com PII
2. Escanear e identificar dados pessoais
3. Classificar por nível de risco
4. Aplicar diferentes técnicas de anonimização
5. Validar a eficácia da anonimização

## Conceitos LGPD

- **Dado Pessoal**: Informação relacionada a pessoa natural identificada ou identificável
- **Dado Sensível**: Origem racial, convicção religiosa, opinião política, dado de saúde, biométrico
- **Anonimização**: Utilização de meios técnicos para impossibilitar a identificação do titular

In [ ]:
# Imports
import sys
from pathlib import Path

# Adicionar diretório raiz ao path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from faker import Faker

# Configurações
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Bibliotecas carregadas!")

## 1. Geração de Dados de Exemplo

Vamos criar um dataset fictício simulando dados de uma empresa com diversos tipos de PII.

In [ ]:
# Gerar dados sintéticos
fake = Faker('pt_BR')
np.random.seed(42)

n_records = 100

df = pd.DataFrame({
    'id': range(1, n_records + 1),
    'nome_completo': [fake.name() for _ in range(n_records)],
    'cpf': [fake.cpf() for _ in range(n_records)],
    'rg': [f"{np.random.randint(10000000, 99999999)}-{np.random.randint(0,9)}" for _ in range(n_records)],
    'email': [fake.email() for _ in range(n_records)],
    'telefone': [fake.phone_number() for _ in range(n_records)],
    'endereco': [fake.address().replace('\n', ', ') for _ in range(n_records)],
    'cep': [fake.postcode() for _ in range(n_records)],
    'data_nascimento': [fake.date_of_birth(minimum_age=18, maximum_age=70).strftime('%d/%m/%Y') for _ in range(n_records)],
    'salario': [round(np.random.uniform(1500, 25000), 2) for _ in range(n_records)],
    'cargo': [fake.job() for _ in range(n_records)],
    'departamento': np.random.choice(['TI', 'RH', 'Financeiro', 'Comercial', 'Operações'], n_records),
    'data_admissao': [fake.date_between(start_date='-10y', end_date='today').strftime('%d/%m/%Y') for _ in range(n_records)]
})

print(f"Dataset criado: {len(df)} registros, {len(df.columns)} colunas")
print(f"\nColunas: {df.columns.tolist()}")
df.head()

In [ ]:
# Salvar dados para uso posterior
output_path = Path('../data/input/dados_funcionarios.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)
print(f"Dados salvos em: {output_path}")

## 2. Scan de Dados Pessoais (PII)

In [ ]:
from src.scanners import PIIScanner, ScanResult

# Inicializar scanner
scanner = PIIScanner()

# Executar scan
result = scanner.scan(df, source_name='dados_funcionarios.csv')

print("=" * 60)
print("RESULTADO DO SCAN")
print("=" * 60)
print(f"Fonte: {result.source_name}")
print(f"Linhas: {result.total_rows:,}")
print(f"Colunas analisadas: {result.columns_scanned}")
print(f"PIIs encontrados: {len(result.pii_found)}")

In [ ]:
# Resumo por nível de risco
print("\nResumo por Nível de Risco:")
for level, count in result.risk_summary.items():
    if count > 0:
        emoji = {'critico': '🔴', 'alto': '🟠', 'medio': '🟡', 'baixo': '🟢'}.get(level, '⚪')
        print(f"  {emoji} {level.upper()}: {count} campos")

In [ ]:
# Detalhes dos PIIs encontrados
print("\nDetalhes dos PIIs Encontrados:")
print("-" * 80)

pii_data = []
for pii in result.pii_found:
    print(f"[{pii.risk_level.value.upper():8}] {pii.column:20} | {pii.pii_type.value:15} | {pii.count} ocorrências")
    pii_data.append({
        'coluna': pii.column,
        'tipo': pii.pii_type.value,
        'risco': pii.risk_level.value,
        'ocorrencias': pii.count
    })

df_pii = pd.DataFrame(pii_data)

In [ ]:
# Visualização: PIIs por Nível de Risco
risk_order = ['critico', 'alto', 'medio', 'baixo']
risk_colors = {'critico': '#d62728', 'alto': '#ff7f0e', 'medio': '#ffbb78', 'baixo': '#2ca02c'}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras por risco
risk_counts = df_pii['risco'].value_counts().reindex(risk_order).fillna(0)
colors = [risk_colors[r] for r in risk_counts.index]
axes[0].bar(risk_counts.index, risk_counts.values, color=colors)
axes[0].set_xlabel('Nível de Risco')
axes[0].set_ylabel('Quantidade de Campos')
axes[0].set_title('Campos com PII por Nível de Risco')

# Gráfico de pizza por tipo
type_counts = df_pii['tipo'].value_counts()
axes[1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
axes[1].set_title('Distribuição por Tipo de PII')

plt.tight_layout()
plt.show()

In [ ]:
# Recomendações
print("\nRecomendações de Tratamento:")
print("-" * 60)
for rec in result.recommendations:
    print(f"• {rec}")

## 3. Técnicas de Anonimização

Vamos explorar diferentes técnicas disponíveis.

In [ ]:
from src.anonymizers import DataAnonymizer

anonymizer = DataAnonymizer()

# Exemplo de dados para demonstração
exemplo = df.iloc[0].copy()
print("Dados Originais:")
print(f"  CPF: {exemplo['cpf']}")
print(f"  Nome: {exemplo['nome_completo']}")
print(f"  Email: {exemplo['email']}")
print(f"  Salário: {exemplo['salario']}")

In [ ]:
# Técnica 1: Mascaramento
print("\n=== MASCARAMENTO ===")
print("Oculta parte dos caracteres mantendo formato reconhecível")
print(f"  CPF original:   {exemplo['cpf']}")
print(f"  CPF mascarado:  {anonymizer.mask_value(exemplo['cpf'], visible_chars=3)}")
print(f"  Email original: {exemplo['email']}")
print(f"  Email mascarado: {anonymizer.mask_value(exemplo['email'], visible_chars=3)}")

In [ ]:
# Técnica 2: Hash
print("\n=== HASH (SHA-256) ===")
print("Transforma em hash irreversível, útil para chaves únicas")
print(f"  CPF original: {exemplo['cpf']}")
print(f"  CPF hash:     {anonymizer.hash_value(exemplo['cpf'], truncate=12)}")
print(f"  Nome hash:    {anonymizer.hash_value(exemplo['nome_completo'], truncate=12)}")

In [ ]:
# Técnica 3: Pseudonimização
print("\n=== PSEUDONIMIZAÇÃO ===")
print("Substitui por identificador artificial mantendo consistência")
print(f"  Nome original: {exemplo['nome_completo']}")
print(f"  Pseudônimo:    {anonymizer.pseudonymize(exemplo['nome_completo'], prefix='Funcionario')}")

In [ ]:
# Técnica 4: Generalização
print("\n=== GENERALIZAÇÃO ===")
print("Reduz precisão dos dados")
print(f"  Salário original:    R$ {exemplo['salario']:,.2f}")
print(f"  Salário generalizado: {anonymizer.generalize_numeric(exemplo['salario'], bin_size=5000)}")

In [ ]:
# Técnica 5: Supressão
print("\n=== SUPRESSÃO ===")
print("Remove completamente o dado")
print(f"  Endereço original: {exemplo['endereco'][:50]}...")
print(f"  Endereço suprimido: {anonymizer.suppress_value(exemplo['endereco'])}")

In [ ]:
# Técnica 6: Ruído (para dados numéricos)
print("\n=== RUÍDO ESTATÍSTICO ===")
print("Adiciona variação aleatória mantendo distribuição")
print(f"  Salário original: R$ {exemplo['salario']:,.2f}")
for i in range(3):
    noisy = anonymizer.add_noise(exemplo['salario'], percentage=10)
    print(f"  Com ruído (±10%): R$ {noisy:,.2f}")

## 4. Anonimização Completa do Dataset

In [ ]:
# Configuração de anonimização baseada na análise
config = {
    # Críticos - Hash ou supressão
    'cpf': {'method': 'hash', 'truncate': 12},
    'rg': {'method': 'hash', 'truncate': 12},
    
    # Altos - Pseudonimização ou mascaramento
    'nome_completo': {'method': 'pseudonymize', 'prefix': 'Colaborador'},
    'email': {'method': 'mask', 'visible_chars': 3},
    'telefone': {'method': 'mask', 'visible_chars': 4},
    
    # Médios - Generalização
    'endereco': {'method': 'suppress'},
    'cep': {'method': 'mask', 'visible_chars': 5},
    'data_nascimento': {'method': 'generalize'},  # Mantém só o ano
    
    # Sensíveis - Generalização ou ruído
    'salario': {'method': 'noise', 'percentage': 15}
}

print("Configuração de Anonimização:")
for col, cfg in config.items():
    print(f"  {col}: {cfg['method']}")

In [ ]:
# Aplicar anonimização
df_anon = anonymizer.anonymize_dataframe(df.copy(), config)

print("\nDataset Anonimizado:")
df_anon.head()

In [ ]:
# Comparação: Antes vs Depois
print("\n" + "=" * 80)
print("COMPARAÇÃO: ANTES vs DEPOIS DA ANONIMIZAÇÃO")
print("=" * 80)

cols_comparar = ['nome_completo', 'cpf', 'email', 'telefone', 'salario', 'endereco']

for col in cols_comparar:
    print(f"\n{col.upper()}:")
    print(f"  Antes: {str(df[col].iloc[0])[:60]}")
    print(f"  Depois: {str(df_anon[col].iloc[0])[:60]}")

## 5. Validação da Anonimização

In [ ]:
# Re-escanear dados anonimizados
result_anon = scanner.scan(df_anon, source_name='dados_anonimizados.csv')

print("=" * 60)
print("SCAN DO DATASET ANONIMIZADO")
print("=" * 60)
print(f"PIIs encontrados: {len(result_anon.pii_found)}")
print(f"\nResumo por Nível de Risco:")
for level, count in result_anon.risk_summary.items():
    if count > 0:
        print(f"  {level.upper()}: {count}")

In [ ]:
# Comparação: Riscos Antes vs Depois
print("\n" + "=" * 60)
print("REDUÇÃO DE RISCO")
print("=" * 60)

for level in ['critico', 'alto', 'medio', 'baixo']:
    antes = result.risk_summary.get(level, 0)
    depois = result_anon.risk_summary.get(level, 0)
    reducao = antes - depois
    pct = (reducao / antes * 100) if antes > 0 else 0
    print(f"  {level.upper():8}: {antes} → {depois} ({'-' if reducao > 0 else '+'}{abs(reducao)} | {pct:.0f}% redução)")

In [ ]:
# Verificar utilidade estatística dos dados (exemplo: salário)
print("\n" + "=" * 60)
print("PRESERVAÇÃO DA UTILIDADE ESTATÍSTICA")
print("=" * 60)

print("\nSalário - Estatísticas Descritivas:")
print(f"{'Métrica':<15} {'Original':>15} {'Anonimizado':>15} {'Diferença':>15}")
print("-" * 60)

metricas = ['mean', 'std', 'min', 'max', 'median']
for m in metricas:
    orig = getattr(df['salario'], m)()
    anon = getattr(df_anon['salario'], m)()
    diff = abs(orig - anon) / orig * 100
    print(f"{m:<15} {orig:>15,.2f} {anon:>15,.2f} {diff:>14.1f}%")

In [ ]:
# Visualização: Distribuição Antes vs Depois
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de salários
axes[0].hist(df['salario'], bins=20, alpha=0.5, label='Original', color='blue')
axes[0].hist(df_anon['salario'], bins=20, alpha=0.5, label='Anonimizado', color='orange')
axes[0].set_xlabel('Salário (R$)')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição de Salários: Original vs Anonimizado')
axes[0].legend()

# Boxplot comparativo
data_box = pd.DataFrame({
    'Original': df['salario'],
    'Anonimizado': df_anon['salario']
})
data_box.boxplot(ax=axes[1])
axes[1].set_ylabel('Salário (R$)')
axes[1].set_title('Comparação de Distribuição')

plt.tight_layout()
plt.show()

## 6. Geração de Relatório de Auditoria

In [ ]:
from src.reporters import LGPDReporter

# Gerar relatório HTML
reporter = LGPDReporter()
report_path = reporter.generate_audit_report([result])

print(f"\nRelatório de auditoria gerado: {report_path}")
print("Abra o arquivo HTML no navegador para visualizar o relatório completo.")

In [ ]:
# Salvar dados anonimizados
output_anon = Path('../data/output/dados_funcionarios_anonimizado.csv')
output_anon.parent.mkdir(parents=True, exist_ok=True)
df_anon.to_csv(output_anon, index=False)
print(f"Dados anonimizados salvos: {output_anon}")

## 7. Conclusões e Boas Práticas

### Técnicas Recomendadas por Tipo de Dado

| Tipo de Dado | Risco | Técnica Recomendada |
|--------------|-------|---------------------|
| CPF, CNPJ, RG | Crítico | Hash SHA-256 |
| Nome completo | Alto | Pseudonimização |
| Email | Alto | Mascaramento ou Hash |
| Telefone | Alto | Mascaramento |
| Endereço | Médio | Supressão ou Generalização |
| Data de nascimento | Médio | Generalização (só ano) |
| Salário | Médio | Ruído estatístico |

### Princípios LGPD Aplicados

1. **Minimização**: Coletar apenas dados necessários
2. **Adequação**: Tratamento compatível com a finalidade
3. **Necessidade**: Limitar ao mínimo necessário
4. **Qualidade**: Garantir exatidão e atualização
5. **Segurança**: Medidas técnicas de proteção

### Próximos Passos

- Implementar processo de auditoria periódica
- Automatizar pipeline de anonimização
- Integrar com sistemas de governança de dados
- Documentar bases legais para cada tratamento